In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

def calculate_total_week(date):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 25)
    pd.set_option('display.precision', 3)
    date_extension = # do somecalc to get 170624 from date
    df1 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + str(date_extension) +'.txt')

    #the name of the exit column is weird
    df1.rename(columns={'EXITS                                                               ': 'EXITS'}, inplace=True)
    #subsetting the stations we want
    df=df1[(df1.STATION=='TIMES SQ-42 ST')| (df1.STATION=='FLUSHING-MAIN' )| (df1.STATION=='ATL AV-BARCLAY')]

    #start of analysis for weekly counts##
    station_date=df.groupby(['C/A','UNIT','SCP','STATION','DATE'])
    entry_date_count=station_date.ENTRIES.apply(lambda x: x.max() - x.min())
    entry_date_count=entry_date_count.reset_index()


    exit_date_count=station_date.EXITS.apply(lambda x: x.max() - x.min())
    exit_date_count=exit_date_count.reset_index()



    #changing the dates to datetime for plotting
    import matplotlib.pyplot as plt
    import dateutil.parser
    dates=[]
    for i in range(0,len(entry_date_count)):
        dates.append(dateutil.parser.parse(entry_date_count.iloc[i,4]))
    entry_date_count['DATE_NEW']=dates



    #group all entries by station#
    station_entry_week_count=entry_date_count.groupby('STATION').ENTRIES.agg(['sum'])

    station_exit_week_count=exit_date_count.groupby('STATION').EXITS.agg(['sum'])


    #total counts
    station_entry_week_count['exit sum']=station_exit_week_count['sum']
    station_entry_week_count['total']=station_entry_week_count['exit sum']+station_entry_week_count['sum']



In [2]:
station_entry_week_count

,sum,exit sum,total
STATION,,,
ATL AV-BARCLAY,231072,223384,454456
FLUSHING-MAIN,330041,236095,566136
TIMES SQ-42 ST,5969841,1681811,7651652
